# 15016c/d: Assessment and Placement / Corequisite Reforms in CCCs

## Investigating English Course Data
#### _Programmer_: Bonnie Brooks

**Purpose**: To compare the list of English Composition courses we have available in the latest download of the California Community College Chancellor's Office Management Information System (COMIS) to the list of designated transfer courses provided by the Course Identification Numbering System (C-ID) that are commonly articulated between the the CCC and universities (namely UC, CSU, and other independent colleges/universities). This code is used to identify courses that are and are not present in both COMIS and CID course data
  
**Deliverables**: .csv/.dta files of courses that are present in both datasets; .csv/.dta files of courses that are not present in both datasets

**Note**:
- You can only run this Jupyter notebook in the environment where COMIS course data lives (cb_course.dta)

In [ ]:
import pandas as pd
import numpy as np
import sys

pd.set_option('precision', 0)
pd.set_option('display.max_columns', 550)
pd.set_option('display.max_rows', 550)

In [ ]:
df_cid = pd.read_csv("CID_ENGL_2020.csv")
df_cap = pd.read_csv("CAP_COURSES_2019.csv")
df_cb_courses = pd.read_stata("Z:\\projects\\CCCCO\\Data\\Download2020\\StataDatasets2020\\cb_course.dta")

In [ ]:
df_cb_courses.head()

In [ ]:
df_cid.columns

In [ ]:
df_cap.head()

In [ ]:
df_cap = df_cap.drop(['Unnamed: 0'], axis=1)

**_Creating sub-dataframes for courses in COMIS and courses in CID:_**

In [ ]:
df_cap_comp = df_cap[df_cap['Freshman_composition']==1]

In [ ]:
df_cap_comp.head()

In [ ]:
df_cb_courses['counter'] = 1

In [ ]:
#creating a "collapsed" version of cb_course.dta unique to COLLEGE_NAME, CONTROL_NUMBER, COURSE_ID, TOP_CODE

#STATA: collapse (<stat>) <stat_vars>, by(<groupvars>)
#PYTHON: df.groupby(<groupvars>)[<stat_vars>].<stat>().reset_index()

cb_courses_collapsed = df_cb_courses.groupby(['COLLEGE_NAME', 'CONTROL_NUMBER', 'COURSE_ID', 'TOP_CODE'])['counter'].sum().reset_index()
cb_courses_collapsed = cb_courses_collapsed[(cb_courses_collapsed['TOP_CODE']=="150100") | (cb_courses_collapsed['TOP_CODE']=="152000")]

In [ ]:
cb_courses_collapsed.head()

##### _**Problem:**_ 
  
Sub-dataframes of CID, COMIS, and cb_course don't have the same spelling of course id numbers across each dataset. 

##### **_Solution:_** 
  
Need to create a function that cleans the COURSE_ID column in each respective dataframe and eliminates inconsistiencies in spelling, spacing, etc. 

In [ ]:
dfs = [df_cid, df_cap_comp, cb_courses_collapsed]

for df in dfs:
    df['COLLEGE_NAME'] = df['COLLEGE_NAME'].str.strip()
    df['Alpha'] = df['COURSE_ID'].str.extract('([a-zA-Z]+)', expand=True)
    df['Numeric'] = df['COURSE_ID'].str.extract('([0-9]+\S\d+|[0-9]\w{0,}[A-Z]|[0-9])', expand=True) # extracting numeric characters from COURSE_ID 
    df['COURSE_ID_NEW'] = df['Alpha'] + "_" + df['Numeric']
    df['COURSE_ID_NEW'] = df['COURSE_ID_NEW'].str.upper().str.strip().str.replace("-", "")
    

In [ ]:
cb_courses_collapsed.head()

In [ ]:
# merging COMIS and CID dataframes, flagging which courses are and aren't in both datasets

df_merged = pd.merge(df_cap_comp, df_cid[['COLLEGE_NAME', 'COURSE_ID_NEW', 'Local Course Title(s)', 'Approval date', 'COR effective term']], how='outer', on=['COLLEGE_NAME', 'COURSE_ID_NEW'], indicator=True)
df_merged = df_merged.sort_values(by=['COLLEGE_NAME', 'COURSE_ID_NEW'])

In [ ]:
df_merged.head()

In [ ]:
df_merged._merge.value_counts()

In [ ]:
df_merged.to_csv("CAP_CID_MERGE.csv")

**_We're also interested in seeing which courses from the CID that weren't in COMIS are present in our most up-to-date download of the COMIS course data (cb-course):_**

In [ ]:
cid_not_in_cap = df_merged[df_merged['_merge']=='right_only']

In [ ]:
cid_not_in_cap.head()

In [ ]:
cid_cb_courses = cid_not_in_cap.merge(cb_courses_collapsed[['COLLEGE_NAME', 'COURSE_ID_NEW']], how="outer", on=['COLLEGE_NAME', 'COURSE_ID_NEW'],  indicator='still_missing')

In [ ]:
cid_cb_courses = cid_cb_courses[['COLLEGE_NAME', 'COURSE_ID_NEW', 'Freshman_composition', 'Local Course Title(s)', 'Approval date', 'COR effective term', 'still_missing']]

In [ ]:
cid_cb_courses.head()

In [ ]:
cid_cb_courses.still_missing.value_counts()

In [ ]:
cid_cb_courses.to_csv("CID_NOT_IN_CAP_MERGED_WITH_CB_COURESES.csv")